In [ ]:
import math
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np

from pykern.pkcollections import PKDict
# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    print("The rslaser library was not found; installing from GitHub...")
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser
from rslaser.rsoptics import wavefront
from rslaser.rspulse import pulse

import scipy
import scipy.constants as const

# SRW and srwlib (the corresponding Python API) are pre-installed on Sirepo/Jupyter
import srwlib
from srwlib import srwl

In [ ]:
# specify all default values of system parameters in a Pykern dictionary object

# photon energy corresponding to lambda = 1 micron
_LAMBDA_DEFAULT = 1.e-6
_PHE_DEFAULT = const.h * const.c / _LAMBDA_DEFAULT
_Z_WAIST_DEFAULT = 0
_Z_CENTER_DEFAULT = 100.*_LAMBDA_DEFAULT

_LASER_PULSE_SLICE_DEFAULTS = PKDict(
    sigrW=0.000186,
    propLen=15,
    pulseE=0.001,
    poltype=1,
    sampFact=5,
    numsig=3.,
    mx=0,
    my=0
)
_LASER_PULSE_DEFAULTS = PKDict(
        phE=_PHE_DEFAULT,
        nslice=3,
        chirp=0,
        w0=.1,
        a0=.01,
        dw0x=0.0,
        dw0y=0.0,
        z_waist=_Z_WAIST_DEFAULT,
        dzwx=0.0,
        dzwy=0.0,
        tau_fwhm=0.1 / const.c / math.sqrt(2.),
        z_center=_Z_CENTER_DEFAULT,
        x_shift = 0.,
        y_shift=0.,
        d_to_w=_Z_WAIST_DEFAULT - _Z_CENTER_DEFAULT,
        slice_params=_LASER_PULSE_SLICE_DEFAULTS,
)

In [ ]:
# the laser pulse parameters are loaded into a Pykern dictionary object
# the slice parameters are included hierarchically
in_pulse = PKDict(
    **_LASER_PULSE_DEFAULTS
    )

# Specify non-default values for laser pulse parameters at center of crystal
in_pulse.w0 = 400.e-6         # waist size [m]   

In [ ]:
# instantiate the laser pulse (this can take a few seconds)
LP = pulse.LaserPulse(in_pulse)

In [ ]:
# choose one of the laser pulse slices, and grab its SRW wavefront object
slice=LP.slice
wfr0=slice[1].wfr

In [ ]:
#Plot initial wavefront
##Initial wfr calculation
arIinit = srwlib.array('f', [0]*wfr0.mesh.nx*wfr0.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIinit, wfr0, 6, 0, 3, wfr0.mesh.eStart, 0, 0) #extracts intensity

##Reshaping electric field data from flat to 2D array
arIinit2D = np.array(arIinit).reshape((wfr0.mesh.nx, wfr0.mesh.ny), order='C')
wfrsizei=np.size(arIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvalsf02=np.linspace(wfr0.mesh.xStart,wfr0.mesh.xFin,wfr0.mesh.nx)
yvalsf02=np.linspace(wfr0.mesh.yStart,wfr0.mesh.yFin,wfr0.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

# By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
plt.pcolormesh(np.multiply(xvalsf02,1e6), np.multiply(yvalsf02,1e6), arIinit2D, cmap=plt.cm.viridis,shading='auto')
plt.colorbar()
#['Horizontal Position [mm]', , 'Intensity']
# ax.set_ylim(-1e4,1e4)
# ax.set_xlim(-1e4,1e4)
ax.set_ylabel(r'Vertical Position [$\mu m$]')
ax.set_xlabel(r'Horizontal Position [$\mu m$]')
ax.set_title('Intensity')
plt.show()

In [ ]:
# create a Gaussian wavefront via SRW
# is w0 the same as sigrW ...
propLen = 0.
pulseE = 1.
poltype = 0
wfr0=wavefront.createGsnSrcSRW(in_pulse.w0,propLen,pulseE,poltype,in_pulse.phE)